<a href="https://colab.research.google.com/github/Kavyapm1960/project/blob/main/eng_to_fr_trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
def load_data(filename):
    with open(filename, encoding='utf-8') as file:
        text = file.read()
    return text.split('\n')

def tokenize(sentences):
    tokenizer = Tokenizer(char_level=False)
    tokenizer.fit_on_texts(sentences)
    return tokenizer, tokenizer.texts_to_sequences(sentences)

def pad(sequences, maxlen=None):
    return pad_sequences(sequences, padding='post', maxlen=maxlen)

english_sentences = load_data('/content/drive/MyDrive/small_vocab_en.txt')
french_sentences = load_data('/content/drive/MyDrive/small_vocab_fr.txt')

english_tokenizer, english_tokenized = tokenize(english_sentences)
french_tokenizer, french_tokenized = tokenize(french_sentences)

max_english_length = max(len(sentence) for sentence in english_tokenized)
max_french_length = max(len(sentence) for sentence in french_tokenized)

english_padded = pad(english_tokenized, maxlen=max_english_length)
french_padded = pad(french_tokenized, maxlen=max_french_length)

In [4]:
encoder_input_data = np.array(english_padded)
decoder_input_data = np.array(french_padded)
decoder_output_data = np.zeros_like(decoder_input_data)
decoder_output_data[:, :-1] = decoder_input_data[:, 1:]

embedding_size = 256
lstm_units = 512
num_encoder_tokens = len(english_tokenizer.word_index) + 1
num_decoder_tokens = len(french_tokenizer.word_index) + 1

encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding_layer = Embedding(num_decoder_tokens, embedding_size)
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


In [5]:

sorted_indices = np.argsort([len(s) for s in english_tokenized])
english_sorted = np.array(english_padded)[sorted_indices]
french_sorted = np.array(french_padded)[sorted_indices]

In [7]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

decoder_output_one_hot = np.zeros(
    (len(french_sentences), max_french_length, num_decoder_tokens),
    dtype='float32'
)

for i, sentence in enumerate(decoder_output_data):
    for t, word_idx in enumerate(sentence):
        if word_idx > 0:
            decoder_output_one_hot[i, t, word_idx] = 1

model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_output_one_hot,
    batch_size=512,
    epochs=80,
    validation_split=0.2
)

Epoch 1/80
216/216 [==============================] - 26s 96ms/step - loss: 2.1563 - accuracy: 0.0927 - val_loss: 1.9156 - val_accuracy: 0.1160
Epoch 2/80
216/216 [==============================] - 16s 76ms/step - loss: 1.7519 - accuracy: 0.1456 - val_loss: 1.5962 - val_accuracy: 0.1819
Epoch 3/80
216/216 [==============================] - 16s 73ms/step - loss: 1.5248 - accuracy: 0.1944 - val_loss: 1.4157 - val_accuracy: 0.2081
Epoch 4/80
216/216 [==============================] - 16s 72ms/step - loss: 1.3493 - accuracy: 0.2374 - val_loss: 1.2694 - val_accuracy: 0.2536
Epoch 5/80
216/216 [==============================] - 16s 73ms/step - loss: 1.2248 - accuracy: 0.2649 - val_loss: 1.2023 - val_accuracy: 0.2627
Epoch 6/80
216/216 [==============================] - 17s 78ms/step - loss: 1.1329 - accuracy: 0.2820 - val_loss: 1.0900 - val_accuracy: 0.2886
Epoch 7/80
216/216 [==============================] - 17s 78ms/step - loss: 1.0647 - accuracy: 0.2964 - val_loss: 1.0575 - val_accuracy:

In [8]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_embedding = decoder_embedding_layer(decoder_inputs)

decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    # Start the target sequence with the first word index or another suitable starting point
    target_seq = np.zeros((1, 1))

    # You might want to start with the most common word or another heuristic
    # For simplicity, we start with the first word index; adjust as needed
    start_token_index = 1  # or another index that makes sense for your dataset
    target_seq[0, 0] = start_token_index

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_french_tokenizer.get(sampled_token_index, '')
        if sampled_word == '<end>' or len(decoded_sentence) > max_french_length:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word
            target_seq = np.zeros((1, 1))
            target_seq[0, 0] = sampled_token_index
            states_value = [h, c]
    return decoded_sentence.strip()

reverse_french_tokenizer = {i: word for word, i in french_tokenizer.word_index.items()}


In [9]:
while True:
    input_sentence = input("Enter English sentence to translate (or type 'quit' to exit): ")
    if input_sentence.lower() == 'quit':
        break
    input_seq = english_tokenizer.texts_to_sequences([input_sentence])
    padded_input_seq = pad(input_seq, maxlen=max_english_length)
    translation = decode_sequence(padded_input_seq)
    print("French translation:", translation)

Enter English sentence to translate (or type 'quit' to exit): the
1/1 [==============================] - 0s 19ms/step
French translation: la poire en février en
Enter English sentence to translate (or type 'quit' to exit): happy birthday
1/1 [==============================] - 0s 19ms/step
French translation: la poire en février en
Enter English sentence to translate (or type 'quit' to exit): the pear in February in
1/1 [==============================] - 0s 18ms/step
French translation: détestons les poires les
Enter English sentence to translate (or type 'quit' to exit): we hate pears
1/1 [==============================] - 0s 18ms/step
French translation: citrons en février en
Enter English sentence to translate (or type 'quit' to exit): February
1/1 [==============================] - 0s 19ms/step
French translation: la poire en février en
Enter English sentence to translate (or type 'quit' to exit): india is hot in may
1/1 [==============================] - 0s 18ms/step
French transla